In [55]:
import numpy as np
import pandas as pd
from openrouteservice import distance_matrix
from openrouteservice import client
import itertools
import time

### Helper Functions
- `generate_distance_matrix`: returns distance matrix
- To get a new api_key, go here: https://openrouteservice.org/

In [107]:
""" Expects a dataframe with n facilities, columns Facility ID, Longitude, and Latitude. Returns n x n row dataframe """
""" Can change profile based on open street maps"""
""" Return output as long or wide. If long, it will be a dataframe. """
""" If it's wide, it returns a dictionary with wide distance and time matrix"""

def generate_distance_matrix(df, 
                       api_key = "5b3ce3597851110001cf62487ed7ac9b82e24fa3b7f2a835b479844c",
                       profile = "driving-car",
                       long = True):
    api_limit = 3500
    clnt = client.Client(key=api_key)
    final_matrix = pd.DataFrame()
    df = df.reset_index(drop=True) # using index to iterate
    
    seg=list(range(0, int(np.floor(api_limit/df.shape[0])))) if df.shape[0] ** 2 > api_limit else list(range(df.shape[0]))

    # Create batch groups
    for i in range(0, df.shape[0], len(seg)):
        if len(seg) == 1:
            start_df = pd.DataFrame(df.iloc[i]).T
            end_df = df.drop(i)
        else:
            end = min(i+len(seg), df.shape[0])
            end_size = i+len(seg) if i+len(seg) < df.shape[0] else df.shape[0] - i
            start_df = pd.DataFrame(df.iloc[i:end])
            end_df = df.drop(list(range(i, end)))
            seg = seg[:end_size]

        
        
        combo_first = pd.concat([start_df, end_df]).reset_index(drop=True)

        locations_df = combo_first[["longitude", "latitude"]].copy()
        locations_list = locations_df.values.tolist()
        request = {'locations': locations_list,
               'profile': profile,
               'metrics':['distance','duration'],'sources':seg,'dry_run':False}
        # Do the request
        matrix = clnt.distance_matrix(**request)
        print("Calculated {}x{} routes.".format(len(matrix['durations']),len(matrix['durations'][0])))
        
        time.sleep(1)
        
        lst = list(start_df["facility_id"])
        matrix_temp = pd.DataFrame({"start": list(itertools.chain.from_iterable(itertools.repeat(x, df.shape[0]) for x in lst)),
              "end": list(combo_first["facility_id"]) * len(seg),
            "Distance (km)": list(itertools.chain(*matrix["distances"])), 
              "Duration (hr)" : list(itertools.chain(*matrix["durations"]))})
        final_matrix = pd.concat([final_matrix, matrix_temp])
        
    final_matrix = final_matrix.reset_index(drop=True)
    final_matrix["Distance (km)"] = final_matrix["Distance (km)"]/1000
    final_matrix["Duration (hr)"] = final_matrix["Duration (hr)"]/3600
    
    if long is False:
        dist = final_matrix.copy()
        final_dict = dict()
        final_dist = pd.DataFrame()
        final_time = pd.DataFrame()
        # This preserves order
        unique_fac = pd.unique(dist["end"])
        for fac in unique_fac:
            dist_temp = pd.DataFrame({str(fac): dist[dist["end"] == fac]["Distance (km)"]}).reset_index(drop=True)
            final_dist = pd.concat([final_dist, dist_temp], axis = 1)
            time_temp = pd.DataFrame({str(fac): dist[dist["end"] == fac]["Duration (hr)"]}).reset_index(drop=True)
            final_time = pd.concat([final_time, time_temp], axis = 1)
        final_dict["time"] = final_time
        final_dict["dist"] = final_dist
        return final_dict
    else:
        # Returns a single matrix
        return final_matrix

In [108]:
og = facility.head(10)

In [110]:
# Returns dataframe
test = generate_distance_matrix(og)

Calculated 10x10 routes.


In [111]:
# Returns dictionary
test = generate_distance_matrix(og, long = False)

Calculated 10x10 routes.


## Update reference table

### Update Facility, Distance, and Time

#### 1. Read in old reference table and new facilities to add to the OD matrix 

In [3]:
path = "/Users/lilianchin/Documents/usaid/zambia/Zambia Hub Analysis/streamlit/psm-distribution-route-optimizer/data/"
filename = "Zambia_Reference.xlsx"
old_facility = pd.read_excel(path + filename, sheet_name = "Facility")

##### Either transform from rfp 

In [4]:
rfp = pd.read_excel("/Users/lilianchin/Documents/usaid/zambia/Zambia Hub Analysis/Client Data/RFP Annex 03_Site List and Distribution Matrix_V3_13JAN2021.xlsx", sheet_name = "Luapula Level 2", header=2)

In [5]:
rfp = rfp[rfp["GPS Point"].notna()]
rfp[["latitude", "longitude"]] = rfp["GPS Point"].str.split(",", expand=True)
rfp = rfp.rename(columns = {"Drop off Station/Health Facility/DHO": "facility", "Facility Code": "facility_id"})
rfp = rfp[["facility", "facility_id", "latitude", "longitude"]]
new_facility = rfp[rfp.facility_id.notna()].head(10)

##### Match Facility from master list

In [81]:
gps = pd.read_csv("Zambia Facility GPS Coordinates.csv")

In [82]:
master_list = pd.read_excel("Copy of Final of Master Customer List 26th March 2021  BN.xlsx 2.xlsx", sheet_name = "FINAL", header = 1)


In [83]:
master_list = master_list[["Facility Code", "Facility Name", "Facility Type", "Max Characters(20).7"]]
master_list = master_list[master_list["Facility Code"].notna()]
master_list = master_list.rename(columns = {"Facility Code": "FacilityID", "Facility Type": "type", "Max Characters(20).7": "region"})

In [84]:
master_list["FacilityID"] = master_list["FacilityID"].astype(str)
gps["FacilityID"] = gps["FacilityID"].astype(str)

In [85]:
gps = gps.merge(master_list, on = "FacilityID", how="outer")

In [86]:
print(f'{gps.shape[0]} total facilities. {gps["Final Lat"].isna().sum()} of facilities missing locations. {gps["Facility Name"].isna().sum()} with FacilityID with no facility name')

2531 total facilities. 295 of facilities missing locations. 101 with FacilityID with no facility name


In [87]:
gps = gps.rename(columns = {"FacilityID": "facility_id", "Final Lat": "latitude", "Final Lon": "longitude", "Facility Name": "facility"})

In [88]:
# Grab facilities with complete coordinates
new_facility = gps.dropna(axis=0, how="any")

In [101]:
### Only grab everything in Central and Copper - pull the rest when needed
new_facility = new_facility[new_facility.region.isin(["Central 1", "Central 2", "Central 3", "Central 4", "Central 5", "Central 6", "Central 7"])]

##### or read in excel that has columns: facility, facility_id, latitude, longitude

In [ ]:
# new_facility = pd.read_excel("sample.xlsx")

#### 2. Concatenate to current facility

In [103]:
facility = pd.concat([new_facility, old_facility]).drop_duplicates("facility_id", keep="first")

In [104]:
facility["type"] = facility["type"].fillna("Health Center")

In [105]:
facility = facility.sort_values("facility")

In [106]:
print(f'There are {facility.shape[0]} facilities')

There are 282 facilities


#### 3. Find new distance and time matrix

In [115]:
matrix = generate_distance_matrix(facility, api_key = "5b3ce3597851110001cf62481c3558504d0942afb495c0dbfbbe3584", long=False)

Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 12x282 routes.
Calculated 6x282 routes.


In [118]:
dist = matrix["dist"]

In [119]:
time = matrix["time"]

#### If there are missing distances, remove those facilities from the distance and time matrix, as well as from facilities.

In [132]:
missing_facility_ids = dist.isna().sum()[dist.isna().sum() == dist.shape[0]].index

In [155]:
print(missing_facility_ids)

Index(['103005', '104012', '103026', '1060A9', '803016', '1050C9', '105000',
       '1060F9'],
      dtype='object')


In [141]:
facility = facility[~facility.facility_id.isin(missing_facility_ids)]

In [153]:
dist = dist.dropna(axis=1, how="all").dropna(axis=0, how="all")

In [154]:
time = time.dropna(axis=1, how="all").dropna(axis=0, how="all")

In [156]:
assert(facility.shape[0] == dist.shape[0])

#### Add to existing reference table

In [157]:
# Read in old Reference table
cubage = pd.read_excel(path + filename, sheet_name = "Cubage")
fleet = pd.read_excel(path + filename, sheet_name = "Fleet")
fleet_exclusions = pd.read_excel(path + filename, sheet_name = "Fleet Exclusions")
facility_groups = pd.read_excel(path + filename, sheet_name = "Facility Groups")
distance_adj = pd.read_excel(path + filename, sheet_name = "Distance Adj")
parameters = pd.read_excel(path + filename, sheet_name = "Parameters")

In [158]:
writer = pd.ExcelWriter("Zambia_Reference.xlsx", engine='xlsxwriter')
facility.to_excel(writer, sheet_name = "Facility", index=False)
dist.to_excel(writer, sheet_name = "Distance", index=False)
time.to_excel(writer, sheet_name = "Time", index=False)
cubage.to_excel(writer, sheet_name = "Cubage", index=False)
fleet.to_excel(writer, sheet_name = "Fleet", index=False)
fleet_exclusions.to_excel(writer, sheet_name = "Fleet Exclusions", index=False)
facility_groups.to_excel(writer, sheet_name = "Facility Groups", index=False)
distance_adj.to_excel(writer, sheet_name = "Distance Adj", index=False)
parameters.to_excel(writer, sheet_name = "Parameters", index=False)
writer.save()

-------------------------------------------------------------------------------------------------------------------

### To-do:


1. With facility lists, check if there are new facilities that exist in OD pair. Drop it if it exists
2. Get a new query for ones that aren't in the distance matrix (Keep track of update time)
3. Only do facilities for certain hubs 

This would require updating with just new_facilities x old_facilities instead of updating everything. 



In [ ]:
# INCOMPLETE
def generate_nxm_distance_matrix(new_df, old_df,
                       api_key = "5b3ce3597851110001cf62487ed7ac9b82e24fa3b7f2a835b479844c",
                       profile = "driving-car",
                       long = True,
                       distance = True):
    api_limit = 3500
    clnt = client.Client(key=api_key)
    final_matrix = pd.DataFrame()
#     df = df.reset_index(drop=True) # using index to iterate
    
#     seg=list(range(0, int(np.floor(api_limit/df.shape[0])))) if df.shape[0] ** 2 > api_limit else list(range(df.shape[0]))

#     # Create batch groups
#     for i in range(0, df.shape[0], len(seg)):
#         if len(seg) == 1:
#             start_df = pd.DataFrame(df.iloc[i]).T
#             end_df = df.drop(i)
#         else:
#             end = min(i+len(seg), df.shape[0])
#             end_size = i+len(seg) if i+len(seg) < df.shape[0] else df.shape[0] - i
#             start_df = pd.DataFrame(df.iloc[i:end])
#             end_df = df.drop(list(range(i, end)))
#             seg = seg[:end_size]
    
        
        
#     combo_first = pd.concat([new_df, old_df]).reset_index(drop=True)
#     iAll = list(range(combo_first.shape[0]))
#     seg = iAll[:new_df.shape[0]]
    
    combo_first = pd.concat([new_df, old_df]).reset_index(drop=True)
    iAll = list(range(combo_first.shape[0]))
    seg = iAll[:new_df.shape[0]]
    
    locations_df = combo_first[["longitude", "latitude"]].copy()
    locations_list = locations_df.values.tolist()
    request = {'locations': locations_list,
           'profile': profile,
           'metrics':['distance','duration'],'sources':seg,'dry_run':False}
    # Do the request
    matrix = clnt.distance_matrix(**request)
    print("Calculated {}x{} routes.".format(len(matrix['durations']),len(matrix['durations'][0])))

#     lst = list(start_df["facility_id"])
#     matrix_temp = pd.DataFrame({"start": list(itertools.chain.from_iterable(itertools.repeat(x, df.shape[0]) for x in lst)),
#           "end": list(combo_first["facility_id"]) * len(seg),
#         "Distance (km)": list(itertools.chain(*matrix["distances"])), 
#           "Duration (hr)" : list(itertools.chain(*matrix["durations"]))})
#     final_matrix = pd.concat([final_matrix, matrix_temp])
        
#     final_matrix = final_matrix.reset_index(drop=True)
#     final_matrix["Distance (km)"] = final_matrix["Distance (km)"]/1000
#     final_matrix["Duration (hr)"] = final_matrix["Duration (hr)"]/3600
    
    
            
    return matrix